In [43]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

In [44]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
from utils import rmse
import torch
import numpy as np
import pandas as pd
import altair as alt
import random
import itertools
data_folder = '../local_data'
work_folder = '../local_work'
configs_folder = '../configs'

In [45]:
def denormalize_decode_result(y_hat, max_speed, max_spacing, min_speed, min_spacing):
    """input: 1,14 tensor
        output: 1,3 tensor with wattage no longer one hot encoded"""
    watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

    watt_arg = torch.argmax(y_hat[0][2:])
    y_final = torch.empty(3, dtype=torch.float32)
    y_final[0] = max_speed * y_hat[0][0] # TODO call the scale
    y_final[1] = max_spacing * y_hat[0][1] # TODO call the scale
    y_final[2]= watts[watt_arg]
    return y_final

# Inverse

In [50]:
filepath = Path(f"{work_folder}/saved_best/best_I-cnn-stainless-2022-07-12_20-20.ckpt")#best_I-ann-stainless-2022-07-08_20-10.ckpt") #CHANGEME best_I-cnn-stainless-2022-07-08_16-52.ckpt, best_I-ann-stainless-2022-07-08_20-10.ckpt
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-cnn-stainless-2022-07-12_20-20.ckpt


In [51]:
def get_test_data(path):
    stain_data = torch.load(
                    Path(f"{data_folder}/{path}"))#["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
    wave_data = stain_data["interpolated_wavelength"]
    emiss_data = stain_data["interpolated_emissivity"]#.keys()#["wavelenght"]
    #print(wave_test.max(1), wave_test.min(1)) #2.5002, 11.9479
    #print(emiss_test.max(1)[0])#.max(1), emiss_test.min(1)) #0, 1
    wave_test = wave_data[round(len(wave_data) * .9):]  
    emiss_test = emiss_data[round(len(wave_data) * .9):]
    return wave_test, emiss_test
    
def get_val_data(path):
    stain_data = torch.load(
                    Path(f"{data_folder}/{path}"))#["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
    wave_data = stain_data["interpolated_wavelength"]
    emiss_data = stain_data["interpolated_emissivity"]#.keys()#["wavelenght"]
    #print(wave_test.max(1), wave_test.min(1)) #2.5002, 11.9479
    #print(emiss_test.max(1)[0])#.max(1), emiss_test.min(1)) #0, 1
    wave_test = wave_data[round(len(wave_data) * .8):round(len(wave_data) * .9)]  
    emiss_test = emiss_data[round(len(wave_data) * .8):round(len(wave_data) * .9)]
    return wave_test, emiss_test
#print(max_wave, min_wave)

In [52]:
from src.models import InverseModel

In [53]:
#pass test data into inverse model, get predicted parameters

i_model = InverseModel.load_from_checkpoint(filepath, strict=False)
i_model.eval()

wave_test, emiss_test = get_test_data('stainless-steel-revised-shuffled.pt')
wave_val, emiss_val = get_val_data('stainless-steel-revised-shuffled.pt')
test_y_hats = [] #torch.empty(size = len(emiss_test))
val_y_hats = []
for emiss in emiss_test:
    with torch.no_grad():
        test_y_hats.append(i_model(emiss.reshape(1, 800)))
for emiss in emiss_val:
    with torch.no_grad():
        val_y_hats.append(i_model(emiss.reshape(1, 800)))

# Direct Model

In [41]:
from src.models import DirectModel
direct_filepath = Path(""
    f"{work_folder}/saved_best/best_D-ann-stainless-2022-07-08_02-25.ckpt") #CHANGEME# 
print(direct_filepath)
if not Path.is_file(direct_filepath):
    raise Exception(f'Model file does not exist at {direct_filepath}!')

../local_work/saved_best/best_D-ann-stainless-2022-07-08_02-25.ckpt


In [42]:
d_model = DirectModel.load_from_checkpoint(direct_filepath)
d_model.eval()
emiss_y_hats = []

for x in val_y_hats:#test_y_hats:
    with torch.no_grad():
        emiss_y_hats.append(d_model(x.reshape(1,14)))

In [18]:
#calculate rmse
pred_rmse_vals = []
columns = ["wavelength", "emissivity", "prediction", "example_number"]
viz_coords = pd.DataFrame(columns = columns)
for i in range(len(emiss_test)):
    pred_rmse_vals.append(rmse(emiss_y_hats[i], emiss_test[i].flatten()).item())
    # for j in range(len(wave_test[i].tolist())):
    #     viz_coords.append([wave_test[i].tolist()[j], emiss_test[i].tolist()[j], "True", i])
    #     viz_coords.append([wave_test[i].tolist()[j], emiss_y_hats[i].flatten().tolist()[j], "Prediction", i])
mean_rmse = np.mean(pred_rmse_vals)
stdev = np.std(pred_rmse_vals)
print(f'Direct Filepath: {direct_filepath}')
print(f'Inverse Filepath: {filepath}')
print(f'SD of RMSEs on test data: {stdev}')
print(f'Mean of RMSEs on test data: {mean_rmse}')


/home/spencersong/metamaterials_ai/src/utils/utils.py:43: UserWarning: Using a target size (torch.Size([800])) that is different to the input size (torch.Size([1, 800])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return (torch.nn.functional.mse_loss(pred, target) + epsilon).sqrt()


Direct Filepath: ../local_work/saved_best/best_D-resnet1d-stainless-2022-07-11_02-08.ckpt
Inverse Filepath: ../local_work/saved_best/best_I-ann-stainless-2022-07-08_20-10.ckpt
SD of RMSEs on test data: 0.15429198315775894
Mean of RMSEs on test data: 0.11250693034586068


In [20]:
#histogram
hist_input = pd.DataFrame({"RMSE":pred_rmse_vals})
alt.Chart(hist_input).mark_bar().encode(
    alt.X("RMSE:Q", bin=True),
    y='count()',
).properties(title= 'Test RMSE Scores')


alt.Chart(...)

In [82]:
#get indices for best and worst n examples
n = 9
worst_pred_indices = np.argpartition(pred_rmse_vals, -n)[-n:]
best_pred_indices = np.argpartition(pred_rmse_vals, n)[:n]

In [83]:
#best_pred_indices
worst_pred_indices

array([ 922, 1718,  897,   12,  914,  942, 2898,  887, 1950])

In [84]:
columns = ["wavelength(um)", "emissivity", "prediction", "Test Examples", "RMSE"]
viz_coords = pd.DataFrame(columns = columns)
iters = 2 #len(emiss_test)):
for i in worst_pred_indices:#best_pred_indices, worst_pred_indices, range(len(best_pred_indices)):
    true_list = [wave_test[i].tolist(), emiss_test[i].tolist(), ["True" for x in range(800)], [f'Ex:{i}, RMSE:{np.round(pred_rmse_vals[i], decimals=3)}' for x in range(800)], [f'{np.round(pred_rmse_vals[i], decimals=3)}' for x in range(800)]]
    true_list = list(map(list, zip(*true_list)))
    pred_list = [wave_test[i].tolist(), emiss_y_hats[i].flatten().tolist(), ["Pred" for x in range(800)], [f'Ex:{i}, RMSE:{np.round(pred_rmse_vals[i], decimals=3)}' for x in range(800)], [f'{np.round(pred_rmse_vals[i], decimals=3)}' for x in range(800)]]
    pred_list = list(map(list, zip(*pred_list)))
    viz_coords = viz_coords.append(pd.DataFrame(true_list, columns = columns))
    viz_coords = viz_coords.append(pd.DataFrame(pred_list, columns = columns))

/tmp/ipykernel_4048877/1746111112.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  viz_coords = viz_coords.append(pd.DataFrame(true_list, columns = columns))
/tmp/ipykernel_4048877/1746111112.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  viz_coords = viz_coords.append(pd.DataFrame(pred_list, columns = columns))
/tmp/ipykernel_4048877/1746111112.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  viz_coords = viz_coords.append(pd.DataFrame(true_list, columns = columns))
/tmp/ipykernel_4048877/1746111112.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  viz_coords = viz_coords.append(pd.DataFrame(pred_list, columns = columns)

In [85]:
viz_coords

,wavelength(um),emissivity,prediction,Test Examples,RMSE
0,2.500184,0.774905,True,"Ex:922, RMSE:0.067",0.067
1,2.512008,0.774104,True,"Ex:922, RMSE:0.067",0.067
2,2.523833,0.773412,True,"Ex:922, RMSE:0.067",0.067
3,2.535657,0.772718,True,"Ex:922, RMSE:0.067",0.067
4,2.547481,0.771968,True,"Ex:922, RMSE:0.067",0.067
...,...,...,...,...,...
795,11.900584,0.177987,Pred,"Ex:1950, RMSE:0.103",0.103
796,11.912409,0.177896,Pred,"Ex:1950, RMSE:0.103",0.103
797,11.924233,0.178177,Pred,"Ex:1950, RMSE:0.103",0.103
798,11.936058,0.177184,Pred,"Ex:1950, RMSE:0.103",0.103


In [86]:

#plot differences between predictions and actual!
alt.data_transformers.disable_max_rows()

chart = alt.Chart(viz_coords).mark_line().encode(
    x='wavelength(um)',
    y='emissivity',
    color='prediction',
    text='RMSE'
).facet(facet = 'Test Examples')#, columns = 3)
chart.display()

alt.FacetChart(...)

In [71]:
#visualize predictions vs original